# ReactAgentDebugger Notebook 演示

这个 notebook 演示如何在 Jupyter 环境中使用调试器，包括智能分析功能。

## 1. 环境准备

In [ ]:
# 安装必要的包
!pip install google-generativeai ipywidgets -q

In [ ]:
import os
import sys
from pathlib import Path

# 添加项目路径
project_root = Path(".").resolve().parent
sys.path.insert(0, str(project_root))

# 设置 API 密钥
# 方式1: 直接设置（不推荐）
# os.environ["GEMINI_API_KEY"] = "your-api-key-here"

# 方式2: 从 .env 文件读取（推荐）
from dotenv import load_dotenv
load_dotenv()

# 检查 API 密钥
if not os.getenv("GEMINI_API_KEY"):
    print("⚠️ 请设置 GEMINI_API_KEY 环境变量")
else:
    print("✅ Gemini API 密钥已设置")

## 2. 创建 Agent 和调试器

In [ ]:
from react_agent import GenericReactAgent, ReactAgentConfig, MemoryLevel
from react_agent_debugger_notebook import NotebookReactAgentDebugger, create_notebook_debugger
from react_agent_debugger import StepBreakpoint, StepType, ConditionalBreakpoint

# 创建 Agent
config = ReactAgentConfig(
    work_dir="output/notebook_debug",
    memory_level=MemoryLevel.SMART,
    llm_model="deepseek-chat",
    llm_temperature=0,
    specification="Notebook 调试演示 Agent"
)

agent = GenericReactAgent(config, name="notebook_demo")

# 创建调试器
debugger = create_notebook_debugger(agent)

print("✅ Agent 和调试器创建成功")

## 3. 设置断点

In [ ]:
# 添加各种类型的断点

# 1. 步骤断点 - 在 THINK 步骤暂停
debugger.add_breakpoint(StepBreakpoint("bp_think", StepType.THINK))

# 2. 条件断点 - 当创建 Python 文件时暂停
debugger.add_breakpoint(
    ConditionalBreakpoint(
        "bp_python_file",
        lambda ctx: any(
            tc.get("name") == "write_file" and 
            tc.get("args", {}).get("file_path", "").endswith(".py")
            for tc in ctx.get("tool_calls", [])
        ),
        "创建 Python 文件"
    )
)

# 3. 条件断点 - 执行超过10步时暂停
debugger.add_breakpoint(
    ConditionalBreakpoint(
        "bp_many_steps",
        lambda ctx: len(debugger.execution_history) > 10,
        "执行超过10步"
    )
)

# 列出所有断点
print("已设置的断点：")
for bp in debugger.list_breakpoints():
    print(f"- {bp['id']}: {bp['type']} - {bp.get('condition', bp.get('step_type', ''))}")

## 4. 创建控制面板

In [ ]:
# 创建交互式控制面板
control_panel = debugger.create_control_panel()
display(control_panel)

## 5. 执行调试任务

运行下面的单元格开始调试会话。当触发断点时：
1. 使用控制面板的按钮来控制执行
2. 点击「分析」按钮使用 Gemini 分析当前状态
3. 查看输出了解执行情况

In [ ]:
# 执行一个会触发多个断点的任务
debugger.execute_task("""
请完成以下任务：
1. 创建一个 calculator.py 文件，实现基本的加减乘除功能
2. 创建一个 test_calculator.py 文件，编写测试用例
3. 尝试运行测试（可能会失败）
4. 如果测试失败，分析原因并修复
5. 创建 README.md 说明文件
""")

## 6. 查看分析历史

In [ ]:
# 查看所有分析结果
print(f"共进行了 {len(debugger.analysis_results)} 次分析\n")

for i, result in enumerate(debugger.analysis_results):
    print(f"\n--- 分析 #{i+1} ---")
    print(f"时间: {result.get('timestamp', 'N/A')}")
    print(f"发现 Bug: {result.get('has_bug', 'N/A')}")
    if result.get('has_bug'):
        print(f"严重程度: {result.get('severity', 'N/A')}")
        print(f"Bug 类型: {result.get('bug_type', 'N/A')}")
        print(f"描述: {result.get('description', 'N/A')}")
        print(f"解决方案: {result.get('solution', 'N/A')}")

## 7. 手动触发分析

你也可以在任何时候手动调用分析功能：

In [ ]:
# 手动分析当前状态
analysis_result = debugger.analysis()

# 结果已经在上面的输出中显示
print("\n分析完成！")

## 8. 高级用法：自定义分析条件

In [ ]:
# 创建一个自动分析的断点
def auto_analyze_on_error(ctx):
    """当检测到错误时自动触发分析"""
    if "error" in str(ctx.get("last_message", "")).lower():
        print("\n🚨 检测到错误，自动触发分析...")
        # 注意：这里需要在断点触发后手动调用分析
        return True
    return False

debugger.add_breakpoint(
    ConditionalBreakpoint(
        "bp_auto_analyze",
        auto_analyze_on_error,
        "错误时自动分析"
    )
)

print("✅ 已添加自动分析断点")

## 9. 导出分析报告

In [ ]:
import json
from datetime import datetime

# 生成分析报告
report = {
    "report_time": datetime.now().isoformat(),
    "agent_name": debugger.agent.name,
    "total_steps": len(debugger.execution_history),
    "total_analyses": len(debugger.analysis_results),
    "bugs_found": sum(1 for r in debugger.analysis_results if r.get("has_bug")),
    "analyses": debugger.analysis_results
}

# 保存报告
report_path = Path("output/notebook_debug/analysis_report.json")
report_path.parent.mkdir(parents=True, exist_ok=True)
with open(report_path, "w", encoding="utf-8") as f:
    json.dump(report, f, ensure_ascii=False, indent=2)

print(f"✅ 分析报告已保存到: {report_path}")
print(f"\n报告概要：")
print(f"- 总执行步数: {report['total_steps']}")
print(f"- 分析次数: {report['total_analyses']}")
print(f"- 发现的 Bug 数: {report['bugs_found']}")

## 10. 清理资源

In [ ]:
# 清理工作目录（可选）
import shutil

if input("是否清理工作目录？(y/n): ").lower() == 'y':
    shutil.rmtree("output/notebook_debug", ignore_errors=True)
    print("✅ 工作目录已清理")
else:
    print("工作目录保留")